In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from src.recommenders import MainRecommender
from src.utils import prefilter_items
from src.metrics import map_k, precision_at_k

# Функции из 1-ого вебинара
import os, sys

from pandas.api.types import CategoricalDtype
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

In [183]:
data = pd.read_csv('data/retail_train.csv')
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [184]:
data_train = prefilter_items(data_train, 6000)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [192]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.rename(columns={'item_id': 'actual'}, inplace=True)

In [186]:
recommender = MainRecommender(data_train)

In [187]:
test = pd.read_csv('data/test_users.csv')
test

,user_id
0,1
1,2
2,3
3,6
4,7
...,...
1703,2494
1704,2496
1705,2498
1706,2499


In [193]:
result['als'] = result['user_id'].apply(lambda x: recommender.get_als_recommendations(x))

In [194]:
result['own'] = result['user_id'].apply(lambda x: recommender.get_own_recommendations(x))

In [195]:
result['popular'] = result['user_id'].apply(lambda _: recommender.overall_top_purchases[:5])

In [196]:
result

,user_id,actual,als,own,popular
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082212, 1028166, 1094924, 1062572, 1075074]","[856942, 9297615, 5577022, 1074612, 9655212]","[1082185, 6534178, 1029743, 995242, 1106523]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[883404, 826249, 8090521, 8090537, 907631]","[835476, 921345, 998206, 964594, 864615]","[1082185, 6534178, 1029743, 995242, 1106523]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 878996, 965267, 1024306, 857006]","[13003092, 995598, 1119051, 9911484, 5569792]","[1082185, 6534178, 1029743, 995242, 1106523]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1003188, 853643, 5981267, 835285, 1082185]","[949836, 845814, 1075524, 9338009, 1097544]","[1082185, 6534178, 1029743, 995242, 1106523]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[840361, 1082185, 981760, 916122, 995242]","[1116578, 981660, 969932, 1070803, 1105433]","[1082185, 6534178, 1029743, 995242, 1106523]"
...,...,...,...,...,...
2037,2496,[6534178],"[844179, 1004906, 893868, 973868, 12810393]","[10285187, 900770, 1076744, 992870, 836171]","[1082185, 6534178, 1029743, 995242, 1106523]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[899624, 826249, 1098066, 981760, 963971]","[1066685, 1033913, 896938, 1135834, 1031864]","[1082185, 6534178, 1029743, 995242, 1106523]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[997796, 892844, 1077490, 1136257, 1048563]","[1076580, 1022066, 1100379, 931579, 968759]","[1082185, 6534178, 1029743, 995242, 1106523]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[826249, 883404, 1098066, 907631, 1029743]","[5568964, 833458, 5570048, 889989, 1131312]","[1082185, 6534178, 1029743, 995242, 1106523]"


In [197]:
result.apply(lambda x: map_k(x['popular'], x['actual'], 5), axis=1).mean()

0.14908619000979392

In [198]:
result.apply(lambda x: map_k(x['als'], x['actual']), axis=1).mean()

0.0890267711394055

In [199]:
result.apply(lambda x: map_k(x['own'], x['actual']), axis=1).mean()

0.16524453150506005

In [97]:
als = result.loc[1, 'als']
actual = result.loc[1, 'actual']

In [98]:
map_k(als, actual)

1.0

In [146]:
check = pd.read_csv('data/sample_submission.csv')

In [152]:
' '.join(map(str, result.loc[0, 'own']))

'856942 9297615 991580 1074612 5577022'

In [204]:
data = prefilter_items(data, 8000)

In [205]:
recommender = MainRecommender(data)

In [206]:
test['Predicted'] = test['user_id'].apply(lambda x: ' '.join(map(str, recommender.get_own_recommendations(x))))

In [207]:
test.rename(columns={'user_id': 'UserId'}, inplace=True)

In [208]:
test

,UserId,Predicted
0,1,856942 9297615 1074612 5577022 991580
1,2,1076580 838136 911974 5567582 831125
2,3,835476 921345 998206 932761 920978
3,6,13003092 995598 6979393 5580166 5570685
4,7,949836 845814 840386 1075524 9338009
...,...,...
1703,2494,1033913 828139 8119157 894360 986187
1704,2496,10285187 900770 1076744 992870 836171
1705,2498,1022066 1076580 1100379 931579 968759
1706,2499,5568964 6904613 5570048 889989 1112627


In [209]:
test.to_csv('submission.csv', index=False)

In [147]:
check

,UserId,Predicted
0,1,1082185 981760 995242 1029743 840361
1,2,1082185 981760 995242 1029743 840361
2,3,1082185 981760 995242 1029743 840361
3,6,1082185 981760 995242 1029743 840361
4,7,1082185 981760 995242 1029743 840361
...,...,...
1703,2494,1082185 981760 995242 1029743 840361
1704,2496,1082185 981760 995242 1029743 840361
1705,2498,1082185 981760 995242 1029743 840361
1706,2499,1082185 981760 995242 1029743 840361
